### Install requirements and downoad packages

In [ ]:
! pip install allennlp==0.8.4

In [ ]:
! pip install overrides==3.1.0

In [ ]:
! git clone https://github.com/mayhewsw/pytorch-truecaser pytorch_truecaser

In [ ]:
!wget https://github.com/mayhewsw/pytorch-truecaser/releases/download/v1.0/wiki-truecaser-model-en.tar.gz


In [ ]:
! pip3 uninstall  scikit-learn -y
! pip3 install -i https://pypi.douban.com/simple scikit-learn==0.19.2

In [7]:
from allennlp.predictors.predictor import Predictor
from pytorch_truecaser.mylib import *
from allennlp.models.archival import load_archive

from typing import List

### Truecase Without NER Mayhewsw

In [2]:
def truecase_withoutner_mayhewsw(sentence, archive, predictor):

  out = predictor.predict(sentence)
  outline = predictor.dump_line(out).split('\n')[0]
  sentence_withoutner = '[START] '+ str(outline) +' [END]'
 
  return sentence_withoutner

In [3]:
archive = load_archive("wiki-truecaser-model-en.tar.gz")
predictor = Predictor.from_archive(archive, "truecaser-predictor")

#example truecase_withoutner_mayhewsw
sent = 'Where did roger marquis die'
truecase_withoutner_mayhewsw(sent, archive, predictor)

'[START] Where did Roger Marquis die [END]'

### Truecase With NER Mayhewsw

In [4]:
def truecase_withner_mayhewsw(sentence_withner, sentence_truecase):

    sent_to_change = sentence_withner.replace("  ", " ")
    sent_to_change = sent_to_change.split(' ')
    if '' in sent_to_change:
        sent_to_change.remove('')
    sent_truecase = sentence_truecase.replace("  ", " ")
    sent_truecase = sent_truecase[8:-6].split(' ')
    if '' in sent_truecase:
        sent_truecase.remove('')

    if "[START]  [END]" not in sent_to_change and len(sent_to_change) == len(sent_truecase)+2:
        for m in range(len(sent_to_change)):
            if '[START]' in sent_to_change[m]:
                id_start = m
            elif '[END]' in sent_to_change[m]:
                id_end = m
        for k in range(id_start):
            sent_to_change[k] = sent_truecase[k]
        for l in range(id_start+1, id_end):
            sent_to_change[l] = sent_truecase[l-1]
        for z in range(id_end+1, len(sent_to_change)-1):
            sent_to_change[z] = sent_truecase[z-2]
        return ' '.join(sent_to_change)
    else:
        return ' '.join(sent_truecase)

In [5]:
#example truecase with ner mayhewsw
sent = 'Where did roger marquis die'

#sentence with ner is taken from NerToSentenceInsertion class
sentence_with_ner = "Where did [START] roger marquis [END] die"

sentence_true_case = truecase_withoutner_mayhewsw(sent, archive, predictor)

truecase_withner_mayhewsw(sentence_with_ner, sentence_true_case)

'Where did [START] Roger Marquis [END] die'

### Reranking of final prediction

In [8]:
def predictions_reranking(preds_main: List[str], preds_secondary: List[str]):
    preds_joined = []
    min_index = min(len(preds_main), len(preds_secondary))
    
    #if the length of pred_main and pred_secondary coincide
    for k in range(min_index):
        if preds_main[k] == preds_secondary[k]:
            preds_joined.append(preds_main[k])
        else:
            preds_joined.append(preds_main[k])
            preds_joined.append(preds_secondary[k])
    
    #if pred_main larger than pred_secondary 
    if len(preds_main) > len(preds_secondary):
        diff = len(preds_main)- len(preds_secondary)
        subset = preds_main[-diff:]
        for t in range(len(subset)):
            preds_joined.append(subset[t])
    
    #if pred_secondary larger than pred_main
    elif len(preds_main) < len(preds_secondary):
        diff = len(preds_secondary)- len(preds_main)
        subset = preds_secondary[-diff:]
        for t in range(len(subset)):
            preds_joined.append(subset[t])
    
    #delete repated ids

    remove_indices = []
    
    for q in range(len(preds_joined)):
        for p in range(q):
            if preds_joined[q] == preds_joined[p]:
                remove_indices.append(q)
    
    preds_joined_upd = [i for w, i in enumerate(preds_joined) if w not in remove_indices]
    
    return preds_joined_upd

1) Obtain prediction Truecase With NER Mayhewsw

2) Obtain prediction for the Truecase Without NER Mayhewsw

3) Obtain prediction for the initial sentence with class NerToSentenceInsertion

4) Compute function predictions_reranking (Truecase With NER Mayhewsw, Truecase Without NER Mayhewsw )

5) Compute function predictions_reranking (class NerToSentenceInsertion, prediction obtained in step 4)

In [9]:
#Example of reranking
# Results obtained for the sentence 'Where did roger marquis die'

#1
preds_truecase_with_ner = "Q7358590, Q7358592, Q1773375".split(', ')
#2
preds_truecase_without_ner = "Q8012493, Q7358590, Q6628660, Q763848, Q7358592".split(', ')
#3
preds_ner_to_sentence_insertion = "Q7358590, Q8068232, Q2712858, Q6628665".split(', ')
#4

reranking_1step = predictions_reranking(preds_truecase_with_ner, preds_truecase_without_ner)
reranking_2step = predictions_reranking(preds_ner_to_sentence_insertion, reranking_1step)
reranking_2step

['Q7358590',
 'Q8068232',
 'Q8012493',
 'Q2712858',
 'Q7358592',
 'Q6628665',
 'Q1773375',
 'Q6628660',
 'Q763848']

In [10]:
import pickle
import torch
import os
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

import sys
sys.path.insert(0,'../../')
from caches.ner_to_sentence_insertion import NerToSentenceInsertion
from genre.fairseq_model import mGENRE  # pylint: disable=import-error,import-error
from wikidata.wikidata_entity_to_label import WikidataEntityToLabel
from caches.genre import GENREWikidataEntityesCache
from wikidata.wikidata_subgraphs_retriever import SubgraphsRetriever
from wikidata.wikidata_shortest_path import WikidataShortestPathCache
from wikidata.wikidata_label_to_entity import WikidataLabelToEntity
from wikidata.wikidata_redirects import WikidataRedirectsCache
from genre.trie import Trie, MarisaTrie  # pylint: disable=unused-import,import-error

In [21]:
# os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [15]:
with open('../../../genre_requirements/lang_title2wikidataID-normalized_with_redirect.pkl', "rb") as f:
    lang_title_wikidata_id = pickle.load(f)

with open('../../../genre_requirements/titles_lang_all105_marisa_trie_with_redirect.pkl', "rb") as f:
    trie = pickle.load(f)

print("finished loading pkl")

finished loading pkl


In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = mGENRE.from_pretrained('../../../genre_requirements/fairseq_multilingual_entity_disambiguation/').eval()
model.to(device)

genre_entities = GENREWikidataEntityesCache(
    model,
    trie,
    lang_title_wikidata_id,
)

In [31]:
import pandas as pd
# Firstly run prepare_wdsq_and_rubq_datasets_for_el_i62.ipynb
test_df = pd.read_csv(
    './RuBQ_test_part4ner.csv',
)
test_df.head()

,S,Q,Q_with_NER
0,Q8070,What can cause a tsunami?,What Can Cause A [START] Tsunami [END]?
1,Q2222,"Who wrote the novel ""uncle Tom's Cabin""?","Who Wrote The Novel ""[START] Uncle Tom [END]'s..."
2,Q83186,"Who is the author of the play ""Romeo and Juliet""?","Who Is The Author Of The Play ""[START] Romeo A..."
3,Q218,What is the name of the capital of Romania?,What Is The Name Of The [START] Capital Of Rom...
4,Q5928,What instrument did Jimi Hendrix play?,What Instrument Did [START] Jimi Hendrix [END]...


In [37]:
test_df['q_without_ner_mayhewsw'] = test_df['Q'].progress_apply(
    lambda q: truecase_withoutner_mayhewsw(q, archive, predictor)
)

100%|██████████| 1913/1913 [00:56<00:00, 33.82it/s]


In [38]:
test_df['q_with_ner_mayhewsw'] = test_df.progress_apply(
    lambda row: truecase_withner_mayhewsw(row['Q_with_NER'], row['q_without_ner_mayhewsw']),
    axis=1
)

100%|██████████| 1913/1913 [00:00<00:00, 20189.02it/s]


In [39]:
def chunks(lst, size):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), size):
        yield lst[i : i + size]

batch_size = 64
Q_with_NER_entities = []
for batch in tqdm(chunks(test_df['Q_with_NER'].values, batch_size), total=test_df.index.size // batch_size):
    Q_with_NER_entities.extend(genre_entities.sentences_batch_to_entities(batch.tolist()))

test_df['Q_with_NER_entities'] = Q_with_NER_entities

30it [00:00, 4479.34it/s]             


In [40]:
q_without_ner_mayhewsw_entities = []
for batch in tqdm(chunks(test_df['q_without_ner_mayhewsw'].values, batch_size), total=test_df.index.size // batch_size):
    q_without_ner_mayhewsw_entities.extend(genre_entities.sentences_batch_to_entities(batch.tolist()))

test_df['q_without_ner_mayhewsw_entities'] = q_without_ner_mayhewsw_entities

30it [00:00, 2960.34it/s]             


In [41]:
q_with_ner_mayhewsw_entities = []
for batch in tqdm(chunks(test_df['q_with_ner_mayhewsw'].values, batch_size), total=test_df.index.size // batch_size):
    q_with_ner_mayhewsw_entities.extend(genre_entities.sentences_batch_to_entities(batch.tolist()))

test_df['q_with_ner_mayhewsw_entities'] = q_with_ner_mayhewsw_entities

30it [00:00, 10477.03it/s]            


In [42]:

test_df['entities_after_rerank_v1'] = test_df.progress_apply(
    lambda row: predictions_reranking(
        [e['id'] for e in row['Q_with_NER_entities']],
        predictions_reranking(
            [e['id'] for e in row['q_with_ner_mayhewsw_entities']],
            [e['id'] for e in row['q_without_ner_mayhewsw_entities']],
        )
    ),
    axis=1
)

100%|██████████| 1913/1913 [00:00<00:00, 7332.94it/s]


In [43]:
def accuracy_top_k(targets, candidate_lists, top_k=5):
    correct = 0
    for idx in range(len(targets)):
        target = targets[idx]
        candidates = candidate_lists[idx]
        correct += int(target in candidates[:top_k])
    return correct / len(targets)


for ecol in ['Q_with_NER_entities', 'q_without_ner_mayhewsw_entities', 'q_with_ner_mayhewsw_entities', 'entities_after_rerank_v1']:
    targets = test_df['S'].values
    if ecol == 'entities_after_rerank_v1':
        candidate_lists = test_df[ecol].values
    else:
        candidate_lists = test_df[ecol].apply(lambda ents: [e['id'] for e in ents]).values

    for k in range(1, 6):
        acc = accuracy_top_k(
            targets,
            candidate_lists,
            k
        )
        print(f'Preds {ecol} | Accuracy@{k} = ', acc)
    print('')

Preds Q_with_NER_entities | Accuracy@1 =  0.5812859383167799
Preds Q_with_NER_entities | Accuracy@2 =  0.6706743335075798
Preds Q_with_NER_entities | Accuracy@3 =  0.7041296393099843
Preds Q_with_NER_entities | Accuracy@4 =  0.7239937271301621
Preds Q_with_NER_entities | Accuracy@5 =  0.7412441191845269

Preds q_without_ner_mayhewsw_entities | Accuracy@1 =  0.5504443282801882
Preds q_without_ner_mayhewsw_entities | Accuracy@2 =  0.6424464192368008
Preds q_without_ner_mayhewsw_entities | Accuracy@3 =  0.6811291165708312
Preds q_without_ner_mayhewsw_entities | Accuracy@4 =  0.702038682697334
Preds q_without_ner_mayhewsw_entities | Accuracy@5 =  0.7250392054364871

Preds q_with_ner_mayhewsw_entities | Accuracy@1 =  0.5849451123889179
Preds q_with_ner_mayhewsw_entities | Accuracy@2 =  0.6638787245164662
Preds q_with_ner_mayhewsw_entities | Accuracy@3 =  0.6999477260846837
Preds q_with_ner_mayhewsw_entities | Accuracy@4 =  0.7208572922111867
Preds q_with_ner_mayhewsw_entities | Accuracy@5 =

In [44]:
test_df.to_pickle('RuBQ_EL_results.pkl')

In [74]:
entity2label = WikidataEntityToLabel()

for ecol in ['Q_with_NER_entities', 'q_without_ner_mayhewsw_entities', 'q_with_ner_mayhewsw_entities']:
    test_df[ecol+'_view'] = test_df[ecol].apply(
        lambda ents: [
            (e['id'], [t for t in e['texts'] if 'en' in t.split('>>')[-1]])
            for e in ents
        ]
    )

In [78]:
test_df[[
    'Q', 'S', 'Q_with_NER_entities_view',
    'Q_with_NER', 'q_without_ner_mayhewsw', 'q_with_ner_mayhewsw',
    'q_without_ner_mayhewsw_entities_view', 'q_with_ner_mayhewsw_entities_view', 'entities_after_rerank_v1'
]].to_excel('RuBQ_EL_results.xlsx')